## Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

-- tracking server: yes, local server
-- backend store: sqlite database
-- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow

mlflow.set_tracking_uri("http://0.0.0.0:5005")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://0.0.0.0:5005'


In Bash
- mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local --host 0.0.0.0 --port 5005 

In [2]:
from mlflow.entities import ViewType
mlflow.search_experiments(view_type=ViewType.ACTIVE_ONLY)

[<Experiment: artifact_location='/home/ubuntu/mlops_zoomcamp_sam/02-mlflow/examples/artifacts_local/0', creation_time=1685020733247, experiment_id='0', last_update_time=1685020733247, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [4]:
mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/05/25 13:21:30 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/home/ubuntu/mlops_zoomcamp_sam/02-mlflow/examples/artifacts_local/1/463d0f6856764447b4a53519ded032fb/artifacts'


/home/ubuntu/anaconda3/envs/exp_tracking_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
from mlflow.entities import ViewType
mlflow.search_experiments(view_type=ViewType.ACTIVE_ONLY)

[<Experiment: artifact_location='/home/ubuntu/mlops_zoomcamp_sam/02-mlflow/examples/artifacts_local/1', creation_time=1685020890239, experiment_id='1', last_update_time=1685020890239, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops_zoomcamp_sam/02-mlflow/examples/artifacts_local/0', creation_time=1685020733247, experiment_id='0', last_update_time=1685020733247, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://0.0.0.0:5005")
client.search_registered_models()

[]

In [26]:
# Get Run ID
run_info = client.search_runs(experiment_ids="1")[0]
run_id = run_info.info.run_id

mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = 'iris_classifier'
)

Registered model 'iris_classifier' already exists. Creating a new version of this model...
2023/05/25 13:35:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris_classifier, version 1
Created version '1' of model 'iris_classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685021738675, current_stage='None', description='', last_updated_timestamp=1685021738675, name='iris_classifier', run_id='463d0f6856764447b4a53519ded032fb', run_link='', source='/home/ubuntu/mlops_zoomcamp_sam/02-mlflow/examples/artifacts_local/1/463d0f6856764447b4a53519ded032fb/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>